# LT healthcare services

Prepared by [**K.Clemons**](mailto:kimberly.clemons@ext.ec.europa.eu) and [**J.Grazzini**](mailto:jacopo.grazzini@ec.europa.eu) ([_Eurostat_](https://ec.europa.eu/eurostat)).

This notebook illustrates the way to produce *ad-hoc* harmonised data collected from LT national authority. It shows how data can be automatically harmonised, using single-use script-based approachr or a reusable metadata-based approach.

## Setting the environment <a id="environment"></a>

In [1]:
PROJECT = 'basic-services'

import os, sys
import functools

import numpy as np
import pandas as pd

In [2]:
THISDIR = os.getcwd()
DATAPATH = '../../../data/healthcare/raw/' # or '/Users/kimberlyclemons/Downloads/Health/LT/'
# os.chdir(DATAPATH)
try:
    print('%s%s' % (PROJECT,os.path.abspath(DATAPATH).split(PROJECT)[1])) 
except:
    print(os.getcwd())

basic-services/data/healthcare/raw


## Ad-hoc data ingestion, exploration and processing

In [3]:
COUNTRY, LANG = 'Lithuania', 'lt'
IFILE = 'Hospitals_2018.xlsx'

try:
    import xlrd
except ImportError:
    !{sys.executable} -m pip install xlrd

file = os.path.join(DATAPATH, IFILE)
df = pd.read_excel(file, header = 2) 

print(df.columns)

Index(['Code of municipality', 'Municipality', 'ID', 'parent_ID',
       'Code of legal entity',
       'Subordination: 1-national (MoH), 3-municipality, 8-private, 9-other ministries (not MoH)',
       'type_code', 'type_name',
       'Level: 1-national, 2-regional, 3-municipality, 4-nursing, 5-other public and specialized, 6-private',
       'Name', 'Address', 'Number of beds at the end of the 2018'],
      dtype='object')


Clean the dataset, add/rename columns, derive information, *etc...*:

In [4]:
df['country'] = COUNTRY
df['public_private'] = (df['Level: 1-national, 2-regional, 3-municipality, 4-nursing, 5-other public and specialized, 6-private']
                       .apply(lambda x: 'private' if x==6 else 'public')
                       )

df.rename(columns = {'ID':                                    'id', 
                     'Name':                                  'name',
                     #'Address':                               'address',
                     'Number of beds at the end of the 2018': 'cap_beds',
                     'type_name':                             'facility_type'}, 
          inplace = True, errors = 'ignore') 

df['address'] = df[['name', 'Address', 'country']].apply(', '.join, 1)
df['Address'] = df[['Address', 'country']].apply(', '.join, 1) # of use when geocoding fails

df.drop(columns = ['Subordination: 1-national (MoH), 3-municipality, 8-private, 9-other ministries (not MoH)',
                   'Level: 1-national, 2-regional, 3-municipality, 4-nursing, 5-other public and specialized, 6-private',
                   'Code of legal entity', 'Code of municipality'],
        inplace = True, errors = 'ignore')

df.head()

,Municipality,id,parent_ID,type_code,facility_type,name,Address,cap_beds,country,public_private,address
0,Vilnius,1,0,1,general,Vilniaus Universiteto ligoninė Santaros klinikos,"Santariškių 2, Vilnius LT-08661, Lithuania",1356,Lithuania,public,Vilniaus Universiteto ligoninė Santaros klinik...
1,Vilnius,5,1,1,general,Vilniaus Universiteto ligoninės Santaros klini...,"Santariškių 7, Vilnius LT-08406, Lithuania",515,Lithuania,public,Vilniaus Universiteto ligoninės Santaros klini...
2,Vilnius,32,0,1,general,Respublikinė Vilniaus universitetinė ligoninė,"Šiltnamių 29, Vilnius, Lithuania",673,Lithuania,public,"Respublikinė Vilniaus universitetinė ligoninė,..."
3,Vilnius,3,0,1,general,Vilniaus Universiteto ligoninės Žalgirio klinika,"Žalgirio 117, Vilnius, Lithuania",58,Lithuania,public,Vilniaus Universiteto ligoninės Žalgirio klini...
4,Vilnius,10,0,19,psychiatry,Respublikinė Vilniaus psichiatrijos ligoninė,"Parko g. 21, Vilnius, Lithuania",542,Lithuania,public,"Respublikinė Vilniaus psichiatrijos ligoninė, ..."


Build a geocoder/geolocator using the [`geopy`](https://geopy.readthedocs.io/en/stable/#) package, possibly inserting an API key to the geocoding service if needed (`OSM` and `Bing` are considered below):

In [5]:
try:
    import geopy
except ImportError:
    !{sys.executable} -m pip install geopy
finally:
    from geopy import geocoders

key = None
try:
    assert key is not None
except:
    geolocator = functools.partial(geocoders.Nominatim(user_agent = PROJECT).geocode, language = LANG)
else:
    geolocator = geocoders.Bing(user_agent = PROJECT, timeout = 100, api_key = key).geocode

Note some way to circumvent the geocoder limitations (this may not be needed for all geocoders):

In [6]:
print('location: %s' % df.iloc[0]['address'])
try:
    location = geolocator(df.iloc[0]['address'])
    assert location is not None
except:
    try:
        location = geolocator(df.iloc[0]['Address'])
        assert location is not None 
    except:
        pass
    else:
        print('lat/lon coordinates: (%s,%s)' % (location.latitude, location.longitude))
else:
    print('lat/lon coordinates: (%s,%s)' % (location.latitude, location.longitude))

location: Vilniaus Universiteto ligoninė Santaros klinikos, Santariškių 2, Vilnius LT-08661, Lithuania
lat/lon coordinates: (54.75234945,25.277250069302262)


Apply the geocoder to the newly built `'address'` variable

In [7]:
df['__coord__'] = df['address'].apply(geolocator)
if df['__coord__'].isnull().any():
    index = df[df['__coord__'].isnull()].index
    df.loc[index, '__coord__'] = df.loc[index, 'Address'].apply(geolocator)
    
df.head()

,Municipality,id,parent_ID,type_code,facility_type,name,Address,cap_beds,country,public_private,address,__coord__
0,Vilnius,1,0,1,general,Vilniaus Universiteto ligoninė Santaros klinikos,"Santariškių 2, Vilnius LT-08661, Lithuania",1356,Lithuania,public,Vilniaus Universiteto ligoninė Santaros klinik...,"(Santariškių klinikos, 2, Santariškių g., Verk..."
1,Vilnius,5,1,1,general,Vilniaus Universiteto ligoninės Santaros klini...,"Santariškių 7, Vilnius LT-08406, Lithuania",515,Lithuania,public,Vilniaus Universiteto ligoninės Santaros klini...,"(Santariškių klinikų vaikų ligoninė, 7, Santar..."
2,Vilnius,32,0,1,general,Respublikinė Vilniaus universitetinė ligoninė,"Šiltnamių 29, Vilnius, Lithuania",673,Lithuania,public,"Respublikinė Vilniaus universitetinė ligoninė,...",(Respublikinė Vilniaus universitetinė ligoninė...
3,Vilnius,3,0,1,general,Vilniaus Universiteto ligoninės Žalgirio klinika,"Žalgirio 117, Vilnius, Lithuania",58,Lithuania,public,Vilniaus Universiteto ligoninės Žalgirio klini...,"(„Žalgirio“ stadiono dekonstrukcijos aikštelė,..."
4,Vilnius,10,0,19,psychiatry,Respublikinė Vilniaus psichiatrijos ligoninė,"Parko g. 21, Vilnius, Lithuania",542,Lithuania,public,"Respublikinė Vilniaus psichiatrijos ligoninė, ...","(Respublikinė Vilniaus psichiatrijos ligoninė,..."


Extract the latitude/longitude coordinates from the geocoder answers:

In [8]:
df['lat'], df['lon'] = zip(*df['__coord__']
                           .apply(lambda x: (x.latitude, x.longitude) if x != None else (np.nan, np.nan))
                          )
df.drop(columns = ['__coord__', 'Address'], inplace = True, errors = 'ignore')

df.head()

,Municipality,id,parent_ID,type_code,facility_type,name,cap_beds,country,public_private,address,lat,lon
0,Vilnius,1,0,1,general,Vilniaus Universiteto ligoninė Santaros klinikos,1356,Lithuania,public,Vilniaus Universiteto ligoninė Santaros klinik...,54.752349,25.277250
1,Vilnius,5,1,1,general,Vilniaus Universiteto ligoninės Santaros klini...,515,Lithuania,public,Vilniaus Universiteto ligoninės Santaros klini...,54.755024,25.282701
2,Vilnius,32,0,1,general,Respublikinė Vilniaus universitetinė ligoninė,673,Lithuania,public,"Respublikinė Vilniaus universitetinė ligoninė,...",54.668216,25.207738
3,Vilnius,3,0,1,general,Vilniaus Universiteto ligoninės Žalgirio klinika,58,Lithuania,public,Vilniaus Universiteto ligoninės Žalgirio klini...,54.693958,25.290799
4,Vilnius,10,0,19,psychiatry,Respublikinė Vilniaus psichiatrijos ligoninė,542,Lithuania,public,"Respublikinė Vilniaus psichiatrijos ligoninė, ...",54.683846,25.417448


Save the output data:

In [9]:
OFILE = 'LT_geolocated.csv'
df.to_csv(OFILE)

## Automating the production